##  Sentinel-3 SLSTR composites

### Service Definition

In [1]:
service = dict([('title', 'Sentinel-3 SLSTR Level-2 reprojection and tiling'),
                ('abstract', 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass and does the reprojection and tiling'),
                ('id', 'ewf-wfp-03-01-02')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [2]:
input_reference = dict([('identifier', 'input_reference'),
                        ('title', 'Sentinel-3 SLSTR Level-2 (SL_2_LST____ descending pass)'),
                        ('abstract', 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass'),
                        ('value', 'https://catalog.terradue.com/sentinel3/search?format=json&uid=S3B_SL_2_LST____20200613T070835_20200613T071135_20200613T085802_0179_040_063_2880_LN2_O_NR_004'),
                        ('stac:collection', 'input_reference'),
                        ('stac:href', 'catalog.json'),
                        ('max_occurs', '16')])

In [3]:
tiling_level = dict([('identifier', 'tiling_level'),
                ('value', '5'),
                ('title', 'Tiling level'),
                ('abstract', 'Tiling level'),
                ('max_occurs', '1')])


**Data path**

This path defines where the data is staged-in. 

In [4]:
data_path = '/workspace/data/slstr'

In [5]:
input_catalog = '/workspace/data/slstr/catalog.json'

### Workflow

#### Import the packages

In [6]:
import os
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'
import sys
sys.path.append(os.path.join(os.environ['PREFIX'], 'conda-otb/lib/python'))
os.environ['OTB_APPLICATION_PATH'] = os.path.join(os.environ['PREFIX'], 'conda-otb/lib/otb/applications')
os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
import otbApplication
import gdal
from helpers import *
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import shape
import shutil
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType
sys.path.append('.')
from tiling import s3_tiles
import time
    
gdal.UseExceptions()

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

In [9]:
collection = next(cat.get_children())

In [10]:
item = next(collection.get_items())

In [11]:
item.properties['eop:orbitDirection']

'DESCENDING'

In [12]:
if item.properties['eop:orbitDirection'] != 'DESCENDING':
    ciop.log('ERROR','Product cannot be used as input')
    raise Exception('Use products with descending orbit direction')

In [13]:
s3_wkt = shape(item.geometry).wkt

### Import Sentinel-3 SLSTR product

In [14]:
operators = ['Read', 
             'Reproject',
             'Write']

In [15]:
read = dict()

s3_path = item.assets['metadata'].get_absolute_href()

read['file'] =  s3_path
read['formatName'] = 'Sen3'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

write = dict()
write['file'] = 's3_slstr'

In [16]:
snap_graph(os.environ['GPT_BIN'],
           operators,
           Read=read, 
           Reproject=reproject,
           Write=write)

0

In [17]:
def s3_to_tile(input_tif, item, tile):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine("-co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW"))
        
    x_min, y_min, x_max, y_max = tile.tile.bounds

    output_tile_name = '{}_L{}_C{}_R{}'.format(item.id,
                                                    tile.level,
                                                    tile.col,
                                                    tile.row)


    gdal.Translate('tmp_{}.tif'.format(output_tile_name),
                   input_tif,
                   projWin=[x_min, y_max, x_max, y_min],
                   projWinSRS='EPSG:4326')

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name),
                   gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    ds = None

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name))
    ds = gdal.Translate('{}.tif'.format(output_tile_name), ds, options=translate_options)
    ds = None

    band_names = ['LST', 'NDVI', 'Land mask', 'Cloud mask']

    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)

    # update extended area to -10000 instead of 0 as gdal.trasnlate does
    ndvi_data = ds.GetRasterBand(1).ReadAsArray()
    land_data = ds.GetRasterBand(2).ReadAsArray()
    cloud_data = ds.GetRasterBand(3).ReadAsArray()
    
    updated_ndvi_data = np.full(ndvi_data.shape, -10000)
    
    updated_ndvi_data = np.where((land_data == 1) | (cloud_data == 1), 
                                 ndvi_data, -10000)
    
    ds.GetRasterBand(1).WriteArray(updated_ndvi_data)
    
    ds.FlushCache()
    
    ds = None
    
    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)
    
    for index in range(ds.RasterCount):

        srcband = ds.GetRasterBand(index+1)

        srcband.SetDescription(band_names[index])

        if index == 0:
            
            srcband.SetNoDataValue(-10000)
    
    ds.FlushCache()

    ds = None
    
    with open(output_tile_name + '.properties', 'w') as file:

        file.write('NDVI Tile L:{1} C:{2} R:{3} {0}\n'.format(item.id,
                                                              tile.level, 
                                                              tile.col, 
                                                              tile.row))
        
        file.write('date={}/{}\n'.format(item.datetime.strftime('%Y-%m-%dT%H:%M:%sZ'), 
                                         item.datetime.strftime('%Y-%m-%dT%H:%M:%sZ')))
        
        file.write('geometry={0}'.format(tile.tile.wkt))


    for f in ['tmp_{}.tif'.format(output_tile_name), 'tmp_{}.tif.ovr'.format(output_tile_name)]:

        if os.path.exists(f):

            os.remove(f)
    
    return True
    

### Tiling

In [18]:
bands = [os.path.join('s3_slstr.data', '{}.img'.format(band)) for band in ['LST', 'NDVI', 'cloud_in', 'confidence_in']]
        
s3_data = read_s3(bands)
        

In [19]:
ds = gdal.Open(bands[0])

geo_transform = ds.GetGeoTransform()
projection_ref = ds.GetProjectionRef()
    

In [20]:
geo_transform, projection_ref

((33.066989,
  0.008707931666666667,
  -0.0,
  10.440218,
  -0.0,
  -0.008707931666666667),
 'GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137.0,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0.0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433],AXIS["Geodetic longitude",EAST],AXIS["Geodetic latitude",NORTH],AUTHORITY["EPSG","4326"]]')

In [21]:
lst = s3_data[:,:,0]
ndvi = s3_data[:,:,1]
cloud = s3_data[:,:,2]
confidence = s3_data[:,:,3]

In [22]:
mask = get_slstr_confidence_mask('land', confidence)

In [23]:
cloud_mask =  get_slstr_mask('gross_cloud', cloud)

In [24]:
output_name = 'temp.tif'

In [25]:
driver = gdal.GetDriverByName('GTiff')

output = driver.Create(output_name, 
                       lst.shape[1], 
                       lst.shape[0], 
                       4, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(projection_ref)
output.GetRasterBand(1).WriteArray(lst)
output.GetRasterBand(2).WriteArray(ndvi)
output.GetRasterBand(3).WriteArray(mask)
output.GetRasterBand(4).WriteArray(cloud_mask)

output.FlushCache()

output = None

In [26]:
cloud_mask

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [27]:
s3_wkt

'POLYGON ((46.2009 -3.02793, 46.7761 -0.411332, 47.3706 2.24896, 47.9766 4.90844, 48.5959 7.56675, 48.4167 7.60304, 47.9695 7.71615, 47.5213 7.81792, 47.0646 7.92483, 46.6127 8.0229, 46.1713 8.12871, 45.7206 8.232950000000001, 45.2687 8.336499999999999, 44.8093 8.43144, 44.3558 8.53463, 43.9117 8.6351, 43.4579 8.737450000000001, 43.0004 8.840590000000001, 42.5532 8.938940000000001, 42.1 9.029540000000001, 41.6474 9.12933, 41.1906 9.228809999999999, 40.7374 9.32245, 40.2877 9.421709999999999, 39.8325 9.51571, 39.3768 9.61777, 38.9219 9.707509999999999, 38.462 9.80294, 38.0086 9.89739, 37.556 9.98452, 37.1004 10.0824, 36.6453 10.1634, 36.1804 10.2607, 35.7279 10.3508, 35.271 10.4402, 34.7382 7.79994, 34.1902 5.16052, 33.6258 2.52228, 33.0532 -0.07059799999999999, 33.5001 -0.172175, 33.9466 -0.281573, 34.3939 -0.367287, 34.8417 -0.477368, 35.2881 -0.575028, 35.7352 -0.6714020000000001, 36.1792 -0.7790280000000001, 36.6315 -0.887003, 37.0752 -0.986731, 37.5277 -1.08469, 37.9735 -1.18888, 3

In [28]:
tiles = s3_tiles(shape(item.geometry), int(tiling_level['value']))


In [29]:
tiles

,col,level,row,s3_tile,tile
0,39,5,16,"POLYGON ((45 0, 39.375 0, 39.375 5.625, 45 5.6...","POLYGON ((45 0, 45 5.625, 39.375 5.625, 39.375..."
1,40,5,16,"POLYGON ((48.14353523629674 5.625, 47.9766 4.9...","POLYGON ((50.625 0, 50.625 5.625, 45 5.625, 45..."
2,40,5,17,"POLYGON ((45 8.392029773617763, 45.2687 8.3364...","POLYGON ((50.625 5.625, 50.625 11.25, 45 11.25..."
3,40,5,15,"POLYGON ((46.2009 -3.02793, 46.0217 -2.98892, ...","POLYGON ((50.625 -5.625, 50.625 0, 45 0, 45 -5..."
4,39,5,15,"POLYGON ((45 -2.765686898274296, 44.6849 -2.69...","POLYGON ((45 -5.625, 45 0, 39.375 0, 39.375 -5..."
5,38,5,16,"POLYGON ((33.75 3.102842381289875, 34.1902 5.1...","POLYGON ((39.375 0, 39.375 5.625, 33.75 5.625,..."
6,39,5,17,"POLYGON ((39.375 9.618125093427128, 39.3768 9....","POLYGON ((45 5.625, 45 11.25, 39.375 11.25, 39..."
7,38,5,17,"POLYGON ((34.28663597457017 5.625, 34.7382 7.7...","POLYGON ((39.375 5.625, 39.375 11.25, 33.75 11..."
8,37,5,15,"POLYGON ((33.75 -0.233403578275476, 33.5001 -0...","POLYGON ((33.75 -5.625, 33.75 0, 28.125 0, 28...."
9,38,5,15,"POLYGON ((39.375 -1.501673863939529, 39.3135 -...","POLYGON ((39.375 -5.625, 39.375 0, 33.75 0, 33..."


In [30]:
for index, tile in tiles.iterrows():
    
    logging.info('Tile L{} C{} R{}'.format(tile.level,
                                       tile.col,
                                       tile.row))


    s3_to_tile(output_name, item, tile)

In [31]:
logging.info('Clean-up') 
os.remove(output_name)

shutil.rmtree('s3_slstr.data')
os.remove('s3_slstr.dim')

time.sleep(45)

for f in glob.glob('./*.tif.aux.xml'):

    os.remove(f)


NameError: name 'glob' is not defined

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.